In [2]:
# =============================================================================
# 팀장용 레포트 톤 보정 Agent
# =============================================================================

import json
from datetime import datetime
from enum import Enum
from dataclasses import dataclass
from typing import Dict, Any, List
from sqlalchemy import Engine, text
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import re
import sys
import os

In [3]:
from dotenv import load_dotenv

load_dotenv()
# 기존 imports
from sqlalchemy import create_engine, text



# DB 설정
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../..')))
from config.settings import DatabaseConfig

db_config = DatabaseConfig()
DATABASE_URL = db_config.DATABASE_URL
engine = create_engine(DATABASE_URL, pool_pre_ping=True)

# LLM 클라이언트 설정
llm_client = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print(f"LLM Client initialized: {llm_client.model_name}")

LLM Client initialized: gpt-4o-mini


In [4]:


# =============================================================================
# JSON 유틸리티 함수들 (재사용)
# =============================================================================

def extract_value(pattern, text, default=""):
    """정규식으로 값 추출"""
    import re
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else default

def extract_number(pattern, text, default=0):
    """정규식으로 숫자 추출"""
    import re
    match = re.search(pattern, text)
    try:
        return int(match.group(1)) if match else default
    except:
        return default

def extract_float(pattern, text, default=0.0):
    """정규식으로 실수 추출"""
    import re
    match = re.search(pattern, text)
    try:
        return float(match.group(1)) if match else default
    except:
        return default

def safe_json_loads_manager(raw_text: str) -> dict:
    """팀장용 레포트 안전한 JSON 로드"""
    if not raw_text:
        return None
    
    print(f"JSON 파싱 시도 - 데이터 타입: {type(raw_text)}, 길이: {len(str(raw_text))}")
    print(f"첫 200자: {str(raw_text)[:200]}")
    
    try:
        # 1차: 원본 그대로 시도
        result = json.loads(raw_text)
        print("✅ 1차 JSON 파싱 성공")
        return result
    except json.JSONDecodeError as e:
        print(f"1차 JSON 파싱 실패: {e}")
    
    try:
        # 2차: 제어 문자 제거 후 시도
        import re
        # 제어 문자 제거 (탭과 개행은 유지)
        cleaned = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', raw_text)
        result = json.loads(cleaned)
        print("✅ 2차 JSON 파싱 성공 (제어 문자 제거)")
        return result
    except json.JSONDecodeError as e:
        print(f"2차 JSON 파싱 실패: {e}")
    
    try:
        # 3차: 더 강력한 정리 후 시도
        cleaned = raw_text.replace('\r\n', '\n').replace('\r', '\n')
        # ASCII 32 미만 문자 제거 (탭과 개행만 유지)
        cleaned = ''.join(char for char in cleaned if ord(char) >= 32 or char in '\n\t')
        result = json.loads(cleaned)
        print("✅ 3차 JSON 파싱 성공 (강력한 정리)")
        return result
    except json.JSONDecodeError as e:
        print(f"3차 JSON 파싱 실패: {e}")
    
    try:
        # 4차: 문제 문자를 공백으로 대체
        import re
        cleaned = re.sub(r'[\x00-\x1F\x7F]', ' ', raw_text)
        # 연속된 공백을 하나로 줄임
        cleaned = re.sub(r'\s+', ' ', cleaned)
        result = json.loads(cleaned)
        print("✅ 4차 JSON 파싱 성공 (문제 문자 공백 처리)")
        return result
    except json.JSONDecodeError as e:
        print(f"4차 JSON 파싱 실패: {e}")

    # 모든 방법 실패 시
    print("❌ 모든 JSON 파싱 방법 실패")
    return None



# =============================================================================
# 데이터 모델 및 타입 정의
# =============================================================================

class ManagerReportType(Enum):
    MANAGER_QUARTERLY = "manager_quarterly"  # 팀장용 분기별
    MANAGER_ANNUAL = "manager_annual"        # 팀장용 연말

@dataclass
class ManagerReportData:
    """팀장용 레포트 데이터를 담는 클래스"""
    id: int
    report_json: Dict[str, Any]
    report_type: ManagerReportType
    team_name: str
    team_leader: str
    period: str
    table_name: str  # "team_evaluations"

@dataclass
class ManagerToneCorrection:
    """팀장용 톤 보정 결과를 담는 클래스"""
    original_text: str
    corrected_text: str
    report_type: ManagerReportType
    llm_response: str
    processing_time: float

# =============================================================================
# DB 유틸리티
# =============================================================================

def row_to_dict(row) -> Dict:
    """SQLAlchemy Row를 Dictionary로 변환"""
    return dict(row._mapping)

class ManagerDatabaseManager:
    """팀장용 레포트 데이터베이스 관리 클래스"""
    
    def __init__(self, engine: Engine):
        self.engine = engine
    
    def fetch_team_evaluation_reports(self) -> List[Dict]:
        """team_evaluations 테이블에서 팀장용 레포트 조회"""
        with self.engine.connect() as connection:
            query = text("""
                SELECT team_evaluation_id as id, report
                FROM team_evaluations
                WHERE report IS NOT NULL
                ORDER BY team_evaluation_id
            """)
            results = connection.execute(query).fetchall()
            return [row_to_dict(row) for row in results]
    
    def update_team_evaluation_report(self, report_id: int, corrected_report: str) -> bool:
        """team_evaluations 테이블 업데이트"""
        try:
            with self.engine.connect() as connection:
                query = text("""
                    UPDATE team_evaluations 
                    SET report = :corrected_report 
                    WHERE team_evaluation_id = :report_id
                """)
                result = connection.execute(query, {
                    "corrected_report": corrected_report,
                    "report_id": report_id
                })
                connection.commit()
                return result.rowcount > 0
        except Exception as e:
            print(f"team_evaluations 업데이트 오류 (ID: {report_id}): {e}")
            return False

# =============================================================================
# LLM 유틸리티
# =============================================================================

class ManagerLLMClient:
    """팀장용 LLM 클라이언트"""
    
    def __init__(self, llm_client: ChatOpenAI):
        self.client = llm_client
    
    def get_manager_tone_correction_prompt(self, report_type: ManagerReportType) -> str:
        """팀장용 레포트 톤 보정 프롬프트 반환"""
        base_prompt = """
# 팀장용 레포트 톤 보정 프롬프트

## 대상 레포트
* **팀장용 분기별 레포트**: 팀원들의(팀의) 분기 성과 평가
* **팀장용 연말 레포트**: 팀원들의(팀의) 연말 성과 평가

## 레포트 식별 기준
* 팀 기본정보 포함 (팀명, 팀장명)
* 팀 전체 성과 분석 및 팀원별 순위 정보
* 팀 운영, 리스크 관리, 향후 전략 등 관리 관련 섹션

## 톤 보정 규칙

### 1. 호칭 통일
* **통일 기준**: "[이름]님은"으로 일관성 있게 사용
* **수정 대상**:
  * "해당 직원은" → "[이름]님은"
  * "[이름]은" → "[이름]님은"
* **예시**: "김개발은" → "김개발님은"

### 2. 문체 일관성
* **통일 기준**: 격식체(존댓말) 사용
* **수정 대상**:
  * 평어체(~다, ~였다) → 격식체(~습니다, ~였습니다)
  * 모든 문장을 존댓말로 통일
* **예시**: "보였다" → "보였습니다"

### 3. 반복 표현 개선
* **문제**: 동일한 협업 표현의 과도한 반복
* **해결책**: 다양한 표현으로 변경
  * "[이름]님과 [이름]님과의 협업을 통해" →
     * "[이름]님, [이름]님과 협력하여"
     * "팀원들 간 협업으로"
     * "팀 차원의 협력을 통해"
     * "조직적 협업으로"

### 4. 부정적 피드백 전달 방식 개선 (팀장용 특화)
* **원칙**: 관리적 관점에서 객관적이고 전략적인 표현 사용
* **수정 예시**:
  * "실수가 잦고 기술 이해력이 부족" → "업무 정확성 향상을 위한 관리적 지원이 필요한 상황입니다"
  * "무관심한 태도" → "업무 몰입도 제고를 위한 동기부여 방안이 필요합니다"
  * "산만한 모습" → "집중력 향상을 위한 환경 조성이 필요합니다"

### 5. 시간 표현 구체화
* **원칙**: 기본_정보의 업무_수행_기간과 일치시키기
* **수정 예시**:
  * "이번 분기 동안" → JSON의 업무_수행_기간 값 사용
  * "2024년 1분기 동안의 성과" → "2024년 1분기 성과"

## 글자수 제한 규칙

### 팀원별 성과 분석 글자수 제한
* **대상**: `팀원_성과_분석` > `팀원별_기여도` > `기여_내용` 필드
* **제한**: 각 팀원의 기여_내용은 **200자 이내**로 작성
* **적용 방법**:
 - 핵심 성과와 주요 Task 중심으로 요약
 - 구체적인 수치(달성률, 기여도 점수)는 유지
 - 반복적인 설명이나 부연 설명 최소화
 - 향후 개선 방향은 간단히 1-2문장으로 정리

### 글자수 단축 가이드라인
* **유지 우선순위**: 
 1. 구체적 수치 (달성률, 기여도 점수)
 2. 주요 Task명과 핵심 성과
 3. 팀 내 순위 및 상대적 위치
* **단축 대상**:
 - "이러한 분석을 통해", "향후 성과 향상을 위해서는" 등 연결어구
 - 중복되는 수식어나 부연 설명
 - 과도한 상세 분석 내용

### 예시 (단축 전 → 후)
**단축 전** (약 400자):
"김개발(SK0002)님은 현재 3개의 Task에 참여하여 평균 달성률 60.0%와 평균 기여도 40.3점을 기록하고 있습니다. 특히 AI 제안서 기능 고도화 개발에서 50%의 달성률을 보이며 27점의 기여도를 나타냈고, 팀 가동률 모니터링 및 개선 Task에서는 75%의 달성률과 91점의 기여도로 두드러진 성과를 보였습니다..."

**단축 후** (약 180자):
"김개발(SK0002)님은 3개 Task 참여로 평균 달성률 60.0%, 기여도 40.3점을 기록했습니다. AI 제안서 개발(50%, 27점), 가동률 개선(75%, 91점)에서 우수한 성과를 보였으나, 신규 고객 발굴 영역은 개선이 필요합니다."

## 팀장용 레포트 특화 톤

### 핵심 톤 특성
* **관리적·전략적**: 팀 전체 성과 관리 관점
* **의사결정 지원**: 팀장의 의사결정에 도움이 되는 정보 제공
* **객관적·분석적**: 데이터 기반의 객관적 분석
* **리스크 중심**: 문제점과 개선 방안에 초점

### 특화 표현 가이드라인

**1. 팀 성과 분석 표현**
* "팀의 전반적인 성과 분석 결과"
* "조직 운영 관점에서 볼 때"
* "팀 목표 달성을 위한 전략적 관점에서"
* "효율적인 팀 운영을 위해"

**2. 팀원 관리 표현**
* "[이름]님에 대한 관리적 지원 방안"
* "팀원 역량 강화를 위한 전략"
* "인력 운용 최적화 관점에서"
* "팀 리더십 발휘를 위해"

**3. 리스크 관리 표현**
* "팀 운영상 주의가 필요한 사항"
* "조직 효율성 저해 요소"
* "사전 관리가 필요한 리스크"
* "팀 성과 최적화를 위한 개선점"

**4. 전략 제안 표현**
* "향후 팀 운영 전략으로"
* "조직 성과 향상을 위한 방안"
* "팀 역량 강화 전략"
* "효과적인 인력 관리 방안"

**5. 객관적 분석 표현**
* "데이터 분석 결과에 따르면"
* "성과 지표 기준으로 평가할 때"
* "정량적 분석을 통해"
* "객관적 관점에서 판단하면"

## 팀장용 레포트 특별 섹션 톤

### 팀원별 성과 분석
* 개인 격려보다는 팀 내 역할과 기여도 중심으로 서술
* 팀 전체 균형과 시너지 관점에서 평가
* 관리적 조치가 필요한 사항을 명확히 제시

### 리스크 및 개선 방안
* 구체적이고 실행 가능한 관리 방안 제시
* 우선순위와 시급성을 명확히 구분
* 예상 효과와 필요 자원을 객관적으로 서술

### 향후 운영 전략
* 단기/중기/장기 관점의 전략적 로드맵 제시
* 팀 목표와 조직 목표의 연계성 강조
* 측정 가능한 성과 지표와 목표 설정

## 유지해야 할 요소

### 1. 수식어는 그대로 유지
* "진심으로", "매우", "크게", "성공적으로" 등의 수식어는 제거하지 않음
* 감정적 표현과 강조 표현은 보존

### 2. 구체적 수치와 데이터
* 달성률, 기여도, 점수 등의 수치 정보는 정확히 유지
* 분석 내용의 실질적 정보는 변경하지 않음

### 3. 평가 기준과 구조
* JSON의 원본 구조와 항목 구성 유지
* 평가 기준과 해석 방식은 그대로 보존

## 팀장용 레포트 특별 고려사항
* 팀장의 의사결정과 팀 운영에 실질적 도움이 되는 정보 제공
* 개인 감정보다는 조직적 관점에서 객관적 서술
* 문제 상황에 대한 구체적이고 실행 가능한 해결책 제시
* 팀 전체의 성과 최적화를 위한 전략적 관점 유지

## 지시사항
아래 JSON 형태의 팀장용 업무 평가 레포트의 톤을 위 규칙에 따라 보정해주세요.
원본 JSON 구조와 데이터는 유지하되, 텍스트 내용만 보정하여 완전한 JSON 형태로 응답해주세요.
특히 팀원별_기여도의 기여_내용은 반드시 200자 이내로 작성해주세요.
"""
        
        if report_type == ManagerReportType.MANAGER_QUARTERLY:
            base_prompt += "\n이 레포트는 팀장용 분기별 레포트입니다."
        else:
            base_prompt += "\n이 레포트는 팀장용 연말 레포트입니다."
        
        return base_prompt
    
    def correct_tone(self, report_json: Dict[str, Any], report_type: ManagerReportType) -> str:
        """LLM을 사용한 팀장용 톤 보정"""
        system_prompt = self.get_manager_tone_correction_prompt(report_type)
        report_text = json.dumps(report_json, ensure_ascii=False, indent=2)
        
        try:
            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=report_text)
            ]
            
            response = self.client.invoke(messages)
            return response.content
            
        except Exception as e:
            print(f"LLM API 오류: {e}")
            return report_text  # 오류 시 원본 반환

# =============================================================================
# 메인 Agent
# =============================================================================

class ManagerReportToneAgent:
    """팀장용 레포트 톤 보정 Agent"""
    
    def __init__(self, engine: Engine, llm_client: ChatOpenAI):
        self.db_manager = ManagerDatabaseManager(engine)
        self.llm_client = ManagerLLMClient(llm_client)
        self.processing_stats = {
            'total_processed': 0,
            'successful': 0,
            'failed': 0,
            'quarterly_reports': 0,
            'annual_reports': 0,
            'reconstructed': 0
        }

    def load_reports_from_db(self) -> List[ManagerReportData]:
        """DB에서 모든 팀장용 레포트 로드"""
        reports = []
        print("=== 팀장용 레포트 로드 시작 ===")

        try:
            # team_evaluations 테이블에서 레포트 로드
            print("\n📂 팀장용 레포트 로드 중...")
            team_reports = self.db_manager.fetch_team_evaluation_reports()
            print(f"DB에서 {len(team_reports)}개의 팀장용 레포트를 가져왔습니다.")
               
            for report_row in team_reports:
                try:
                    print(f"\n🔄 팀장용 레포트 ID {report_row['id']} 처리 중...")
                
                    # 안전한 JSON 로드 사용
                    report_json = safe_json_loads_manager(report_row['report'])
                    
                    if report_json is None:
                        print(f"❌ JSON 처리 최종 실패, 스킵")
                        continue
                        
                    print(f"✅ JSON 처리 성공")
                
                    report_data = ManagerReportData(
                        id=report_row['id'],
                        report_json=report_json,
                        report_type=self.identify_report_type(report_json),
                        team_name=self.extract_team_name(report_json),
                        team_leader=self.extract_team_leader(report_json),
                        period=self.extract_period(report_json),
                        table_name="team_evaluations"
                    )
                
                    reports.append(report_data)
                    print(f"✅ 팀장용 레포트 ID {report_row['id']} 로드 성공")
                    print(f"   - 팀명: {report_data.team_name}")
                    print(f"   - 팀장: {report_data.team_leader}")
                    print(f"   - 기간: {report_data.period}")
                
                except Exception as e:
                    print(f"❌ 레포트 처리 오류 (팀장용 레포트 ID: {report_row['id']}): {e}")
                    continue

        except Exception as e:
            print(f"❌ DB 로드 중 전체 오류: {e}")
            return []

        print(f"\n📊 팀장용 레포트 로드 완료!")
        print(f"   - 총 로드된 레포트: {len(reports)}개")
        quarterly_count = sum(1 for r in reports if r.report_type == ManagerReportType.MANAGER_QUARTERLY)
        annual_count = sum(1 for r in reports if r.report_type == ManagerReportType.MANAGER_ANNUAL)
        print(f"   - 분기별 레포트: {quarterly_count}개")
        print(f"   - 연말 레포트: {annual_count}개")

        return reports
    
    def identify_report_type(self, report_data: Dict[str, Any]) -> ManagerReportType:
        """레포트 타입 식별"""
        try:
            period = report_data.get('팀_기본정보', {}).get('업무_수행_기간', '')
            
            if '연말' in period:
                return ManagerReportType.MANAGER_ANNUAL
            elif '분기' in period:
                return ManagerReportType.MANAGER_QUARTERLY
            
            return ManagerReportType.MANAGER_QUARTERLY
        except Exception as e:
            print(f"레포트 타입 식별 오류: {e}")
            return ManagerReportType.MANAGER_QUARTERLY
    
    def extract_team_name(self, report_data: Dict[str, Any]) -> str:
        """팀명 추출"""
        try:
            team_info = report_data.get('팀_기본정보', {})
            return team_info.get('팀명') or "알 수 없는 팀"
        except Exception as e:
            print(f"팀명 추출 오류: {e}")
            return "알 수 없는 팀"
    
    def extract_team_leader(self, report_data: Dict[str, Any]) -> str:
        """팀장명 추출"""
        try:
            team_info = report_data.get('팀_기본정보', {})
            return team_info.get('팀장명') or "알 수 없음"
        except Exception as e:
            print(f"팀장명 추출 오류: {e}")
            return "알 수 없음"
    
    def extract_period(self, report_data: Dict[str, Any]) -> str:
        """업무 수행 기간 추출"""
        try:
            return report_data.get('팀_기본정보', {}).get('업무_수행_기간', '기간 미상')
        except Exception as e:
            print(f"기간 추출 오류: {e}")
            return "기간 미상"
    
    def correct_report_tone(self, report_data: ManagerReportData) -> ManagerToneCorrection:
        """LLM을 사용한 팀장용 레포트 톤 보정 실행"""
        start_time = datetime.now()
        
        try:
            original_text = json.dumps(report_data.report_json, ensure_ascii=False, indent=2)
            
            llm_response = self.llm_client.correct_tone(report_data.report_json, report_data.report_type)
            
            try:
                corrected_text = llm_response
                if "```json" in llm_response:
                    corrected_text = llm_response.split("```json")[1].split("```")[0].strip()
                elif "```" in llm_response:
                    corrected_text = llm_response.split("```")[1].strip()
                
                # JSON 유효성 검증
                json.loads(corrected_text)
                print(f"✅ LLM 응답 JSON 파싱 성공")
                
            except (json.JSONDecodeError, IndexError) as e:
                print(f"⚠️ LLM 응답 JSON 파싱 오류: {e}")
                corrected_text = original_text
            
            processing_time = (datetime.now() - start_time).total_seconds()
            
            return ManagerToneCorrection(
                original_text=original_text,
                corrected_text=corrected_text,
                report_type=report_data.report_type,
                llm_response=llm_response,
                processing_time=processing_time
            )
            
        except Exception as e:
            print(f"❌ 톤 보정 중 오류: {e}")
            processing_time = (datetime.now() - start_time).total_seconds()
            original_text = json.dumps(report_data.report_json, ensure_ascii=False, indent=2)
            
            return ManagerToneCorrection(
                original_text=original_text,
                corrected_text=original_text,
                report_type=report_data.report_type,
                llm_response=f"오류 발생: {str(e)}",
                processing_time=processing_time
            )
    
    def save_corrected_report(self, report_data: ManagerReportData, corrected_text: str) -> bool:
        """보정된 레포트를 DB에 저장"""
        try:
            return self.db_manager.update_team_evaluation_report(report_data.id, corrected_text)
        except Exception as e:
            print(f"❌ DB 저장 오류 (ID: {report_data.id}): {e}")
            return False
    
    def process_all_reports(self) -> List[ManagerToneCorrection]:
        """모든 팀장용 레포트 처리 및 DB 저장"""
        try:
            reports = self.load_reports_from_db()
        except Exception as e:
            print(f"❌ 레포트 로드 실패: {e}")
            return []
        
        if reports is None or len(reports) == 0:
            print("⚠️ 처리할 팀장용 레포트가 없습니다.")
            return []
        
        corrections = []
        
        print(f"\n🚀 총 {len(reports)}개의 팀장용 레포트를 LLM으로 처리합니다.")
        
        for i, report_data in enumerate(reports, 1):
            print(f"\n[{i}/{len(reports)}] 처리 중: {report_data.team_name} ({report_data.team_leader}) - {report_data.period}")
            
            try:
                self.processing_stats['total_processed'] += 1
                
                if report_data.report_type == ManagerReportType.MANAGER_QUARTERLY:
                    self.processing_stats['quarterly_reports'] += 1
                else:
                    self.processing_stats['annual_reports'] += 1
                
                correction = self.correct_report_tone(report_data)
                corrections.append(correction)
                
                print(f"  ⏱️  처리 시간: {correction.processing_time:.2f}초")
                
                success = self.save_corrected_report(report_data, correction.corrected_text)
                if success:
                    print(f"  ✅ DB 업데이트 성공")
                    self.processing_stats['successful'] += 1
                else:
                    print(f"  ❌ DB 업데이트 실패")
                    self.processing_stats['failed'] += 1
                
            except Exception as e:
                print(f"  ❌ 처리 오류: {e}")
                self.processing_stats['failed'] += 1
                continue
        
        return corrections
    
    def get_processing_statistics(self) -> Dict[str, int]:
        """처리 통계 반환"""
        return self.processing_stats.copy()
    
    def reset_statistics(self):
        """통계 초기화"""
        self.processing_stats = {
            'total_processed': 0,
            'successful': 0,
            'failed': 0,
            'quarterly_reports': 0,
            'annual_reports': 0,
            'reconstructed': 0
        }
        print("📊 팀장용 Agent 통계가 초기화되었습니다.")
    
    def print_summary(self):
        """처리 결과 요약 출력"""
        stats = self.processing_stats
        success_rate = (stats['successful'] / stats['total_processed'] * 100) if stats['total_processed'] > 0 else 0
        
        print(f"""
📊 ManagerReportToneAgent 처리 완료
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔢 처리 통계:
   - 총 처리된 레포트: {stats['total_processed']}개
   - 성공: {stats['successful']}개
   - 실패: {stats['failed']}개
   - 성공률: {success_rate:.1f}%

📋 레포트 타입별:
   - 팀장용 분기별 레포트: {stats['quarterly_reports']}개
   - 팀장용 연말 레포트: {stats['annual_reports']}개

🔧 JSON 처리:
   - 재구성된 레포트: {stats['reconstructed']}개

🎯 Agent 상태: {'✅ 정상 완료' if stats['total_processed'] > 0 else '⏳ 대기 중'}
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        """)

# =============================================================================
# 실행 함수
# =============================================================================

def main_manager():
    """팀장용 Agent 메인 실행 함수"""
    # Agent 초기화
    manager_agent = ManagerReportToneAgent(engine, llm_client)
    
    # 모든 레포트 처리 및 DB 저장
    print("=== 팀장용 레포트 톤 보정 Agent 실행 ===")
    corrections = manager_agent.process_all_reports()
    
    # 결과 출력
    print(f"\n📊 처리 완료: 총 {len(corrections)}개 팀장용 레포트")
    
    # 상세 통계 출력
    manager_agent.print_summary()
    
    # 평균 처리 시간 계산
    if corrections:
        avg_time = sum(c.processing_time for c in corrections) / len(corrections)
        print(f"\n⏱️ 평균 처리 시간: {avg_time:.2f}초")
        
        # 성공한 톤 보정 개수
        successful_corrections = sum(1 for c in corrections if c.corrected_text != c.original_text)
        print(f"🎯 실제 톤 보정된 레포트: {successful_corrections}개")
    
    return corrections

# =============================================================================
# 테스트 및 검증 함수
# =============================================================================

def test_manager_json_reconstruction():
    """팀장용 JSON 재구성 기능 테스트"""
    print("=== 팀장용 JSON 재구성 기능 테스트 ===")
    
    with engine.connect() as connection:
        query = text("SELECT report FROM team_evaluations WHERE report IS NOT NULL LIMIT 1")
        result = connection.execute(query).fetchone()
        
        if result:
            raw_text = result[0]
            
            # 재구성 테스트
            reconstructed = safe_json_loads_manager(raw_text)
            
            if reconstructed:
                print("✅ 팀장용 JSON 재구성 성공!")
                print(f"재구성된 팀명: {reconstructed.get('팀_기본정보', {}).get('팀명', '불명')}")
                print(f"팀장명: {reconstructed.get('팀_기본정보', {}).get('팀장명', '불명')}")
                team_members = reconstructed.get('팀원별_성과_분석', {}).get('팀원_성과표', [])
                print(f"팀원 수: {len(team_members)}명")
                return True
            else:
                print("❌ 팀장용 JSON 재구성 실패")
                return False
        else:
            print("❌ 테스트 데이터 없음")
            return False

def quick_test_manager():
    """팀장용 Agent 빠른 테스트"""
    print("=== 팀장용 Agent 빠른 테스트 ===")
    
    # JSON 재구성 테스트
    if not test_manager_json_reconstruction():
        print("❌ 기본 기능 테스트 실패")
        return
    
    # Agent 초기화 테스트
    try:
        manager_agent = ManagerReportToneAgent(engine, llm_client)
        print("✅ 팀장용 Agent 초기화 성공")
        
        # 레포트 로드 테스트
        reports = manager_agent.load_reports_from_db()
        print(f"✅ 팀장용 레포트 로드 성공: {len(reports)}개")
        
        if reports:
            print("✅ 모든 기본 기능 정상 작동")
            return True
        else:
            print("⚠️ 처리할 팀장용 레포트가 없음")
            return True
            
    except Exception as e:
        print(f"❌ 팀장용 Agent 테스트 실패: {e}")
        return False



In [5]:
# 사용 방법:
# 1. 빠른 테스트: 
# quick_test_manager()
# 2. 실제 처리: 
main_manager()
# 3. JSON 테스트만: 
# test_manager_json_reconstruction()

=== 팀장용 레포트 톤 보정 Agent 실행 ===
=== 팀장용 레포트 로드 시작 ===

📂 팀장용 레포트 로드 중...
DB에서 2개의 팀장용 레포트를 가져왔습니다.

🔄 팀장용 레포트 ID 2 처리 중...
JSON 파싱 시도 - 데이터 타입: <class 'str'>, 길이: 14405
첫 200자: {
    "기본_정보": {
        "팀명": "W1팀",
        "팀장명": "팀장A",
        "업무_수행_기간": "2024년 2분기"
    },
    "팀_종합_평가": {
        "평균_달성률": 67,
        "유사팀_평균": 85.3,
        "비교_분석": "크게 개선 필요",
1차 JSON 파싱 실패: Invalid control character at: line 272 column 36 (char 13661)
2차 JSON 파싱 실패: Invalid control character at: line 272 column 36 (char 13661)
3차 JSON 파싱 실패: Invalid control character at: line 272 column 36 (char 13390)
✅ 4차 JSON 파싱 성공 (문제 문자 공백 처리)
✅ JSON 처리 성공
✅ 팀장용 레포트 ID 2 로드 성공
   - 팀명: 알 수 없는 팀
   - 팀장: 알 수 없음
   - 기간: 기간 미상

🔄 팀장용 레포트 ID 3 처리 중...
JSON 파싱 시도 - 데이터 타입: <class 'str'>, 길이: 11287
첫 200자: {
    "기본 정보": {
        "팀명": "W1팀",
        "팀장명": "팀장A",
        "업무 수행 기간": "2024년 연말",
        "평가 구분": "연간 최종 평가 (Period 4)"
    },
    "팀 종합 평가": {
        "평균 달성률": 132,
        "유사팀 
1차 JSON 파싱 실패: Inval

[ManagerToneCorrection(original_text='{\n  "기본_정보": {\n    "팀명": "W1팀",\n    "팀장명": "팀장A",\n    "업무_수행_기간": "2024년 2분기"\n  },\n  "팀_종합_평가": {\n    "평균_달성률": 67,\n    "유사팀_평균": 85.3,\n    "비교_분석": "크게 개선 필요",\n    "팀_성과_분석_코멘트": "현재 팀의 성과는 전반적으로 평균적인 수준을 보이고 있으며, 팀 평균 달성률은 80.9%로 나타났습니다. 팀원 기여 분석을 통해 각 구성원의 성과를 살펴보면, AI powered ITS 혁신 부문에서 70%의 달성률을 기록하였고, 이는 비중 35%를 차지하여 팀의 주요 성과 영역 중 하나로 부각되고 있습니다. 또한, Bilable Rate(가동률)은 96%로 매우 높은 수치를 보이며, 이 역시 팀의 성과에 긍정적인 영향을 미치고 있습니다. 매출과 매출이익 부문은 각각 30%의 달성률을 기록하였으며, 이 두 영역은 비중이 각각 15%로 상대적으로 낮은 기여도를 보이고 있습니다. 이러한 성과는 팀의 현재 상태를 반영하며, 팀원 간의 협력과 개별 기여가 조화를 이루고 있음을 나타냅니다. 그러나 전체 평균 달성률이 67.1%로 나타난 점은 향후 개선이 필요한 부분으로, 팀의 지속적인 성장과 발전을 위해 각 성과 영역에서의 집중적인 노력이 요구됩니다."\n  },\n  "팀_업무_목표_및_달성률": {\n    "kpi_목록": [\n      {\n        "팀_업무_목표": "Bilable Rate (가동률)",\n        "kpi_분석_코멘트": "팀원들의 평균 가동률은 82%로, 목표인 85%에 미치지 못하고 있습니다. 김개발님은 가동률 개선을 위한 프로세스를 도입했으나, 이설계님은 프로젝트 리스크 감소에 집중하여 가동률에 직접적인 영향을 미치지 않았습니다. 따라서 팀 전체 KPI 달성률은 96%로 평가됩니다.",\n        "달성률": 96,